In [2]:
from __future__ import division
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import scipy.sparse as sp
import time
import argparse
import numpy as np
import easydict
import math
import networkx as nx
# from .layers import *
# from .models import *
# from .utils import *
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from setuptools import setup
from setuptools import find_packages
# from pygcn.utils import load_data, accuracy
# from pygcn.models import GCN
# from pygcn.layers import GraphConvolution

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models
from skimage.io import imread
from skimage.transform import resize

In [2]:
def age_categorize(age):
    age = (age // 10) * 10
    return age

In [3]:
df = pd.read_excel('/workspace/src/IgAN/brca-clinicalforwiki.xls')
df

,bcr_patient_barcode,age_at_initial_pathologic_diagnosis,ajcc_cancer_metastasis_stage_code,ajcc_neoplasm_disease_lymph_node_stage,ajcc_neoplasm_disease_stage,ajcc_tumor_stage_code,anatomic_organ_subdivision,axillary_lymph_node_stage_method_type,breast_cancer_optical_measurement_histologic_type,breast_cancer_surgery_margin_status,...,lymph_node_examined_count,margin_status,menopause_status,number_of_lymphnodes_positive_by_he,patient_id,pretreatment_history,prior_diagnosis,race,tissue_source_site,vital_status
0,TCGA-E2-A107,54,M0,N2a,Stage IIIA,T3,Right Upper Inner Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],...,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,A107,NO,NO,WHITE,E2,LIVING
1,TCGA-E2-A108,64,M0,N2a,Stage IIIA,T2,Right Upper Outer Quadrant,[Not Available],Infiltrating Ductal,[Not Available],...,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,A108,NO,NO,WHITE,E2,LIVING
2,TCGA-E2-A109,64,M0,N0 (i-),Stage IIA,T2,Right Upper Outer Quadrant,Sentinel node biopsy alone,Infiltrating Ductal,[Not Available],...,6,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,0,A109,NO,NO,WHITE,E2,LIVING
3,TCGA-E2-A10B,67,M0,N1a,Stage IIB,T2,Left Lower Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,28,Close,Post (prior bilateral ovariectomy OR >12 mo si...,2,A10B,NO,NO,WHITE,E2,LIVING
4,TCGA-E2-A10C,54,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,13,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,1,A10C,NO,NO,WHITE,E2,LIVING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,TCGA-AO-A0JF,68,M0,N1a,Stage IIA,T1c,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,14,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,A0JF,NO,NO,WHITE,AO,LIVING
330,TCGA-AO-A0JJ,54,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],...,44,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,A0JJ,NO,NO,WHITE,AO,LIVING
331,TCGA-AO-A0JL,59,M0,N2a,Stage IIIA,T2,Left Upper Outer Quadrant|Left Lower Outer Qua...,Axillary lymph node dissection alone,Infiltrating Ductal,[Not Available],...,26,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,7,A0JL,NO,NO,WHITE,AO,LIVING
332,TCGA-AO-A0JM,40,M0,N1,Stage IIB,T2,Right Lower Inner Quadrant|Right Lower Outer Q...,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],...,23,Negative,Pre (<6 months since LMP AND no prior bilatera...,2,A0JM,NO,NO,[Not Available],AO,LIVING


In [4]:
y = df['vital_status']
print(y)

0      LIVING
1      LIVING
2      LIVING
3      LIVING
4      LIVING
        ...  
329    LIVING
330    LIVING
331    LIVING
332    LIVING
333    LIVING
Name: vital_status, Length: 334, dtype: object


In [5]:
age = df['age_at_initial_pathologic_diagnosis']
age = age_categorize(age)
age

0      50
1      60
2      60
3      60
4      50
       ..
329    60
330    50
331    50
332    40
333    40
Name: age_at_initial_pathologic_diagnosis, Length: 334, dtype: int64

In [6]:
df['age_at_initial_pathologic_diagnosis'] = age
df = df[['age_at_initial_pathologic_diagnosis','ajcc_cancer_metastasis_stage_code','ajcc_neoplasm_disease_lymph_node_stage',
'ajcc_neoplasm_disease_stage','ajcc_tumor_stage_code','anatomic_organ_subdivision','axillary_lymph_node_stage_method_type',
'breast_cancer_optical_measurement_histologic_type','breast_cancer_surgery_margin_status','breast_carcinoma_estrogen_receptor_status',
'breast_carcinoma_progesterone_receptor_status','breast_carcinoma_surgical_procedure_name','her2_immunohistochemistry_level_result',
'lymph_node_examined_count','margin_status','menopause_status','number_of_lymphnodes_positive_by_he','pretreatment_history','prior_diagnosis','race']]
df

,age_at_initial_pathologic_diagnosis,ajcc_cancer_metastasis_stage_code,ajcc_neoplasm_disease_lymph_node_stage,ajcc_neoplasm_disease_stage,ajcc_tumor_stage_code,anatomic_organ_subdivision,axillary_lymph_node_stage_method_type,breast_cancer_optical_measurement_histologic_type,breast_cancer_surgery_margin_status,breast_carcinoma_estrogen_receptor_status,breast_carcinoma_progesterone_receptor_status,breast_carcinoma_surgical_procedure_name,her2_immunohistochemistry_level_result,lymph_node_examined_count,margin_status,menopause_status,number_of_lymphnodes_positive_by_he,pretreatment_history,prior_diagnosis,race
0,50,M0,N2a,Stage IIIA,T3,Right Upper Inner Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],Positive,Negative,Simple Mastectomy,2+,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,NO,NO,WHITE
1,60,M0,N2a,Stage IIIA,T2,Right Upper Outer Quadrant,[Not Available],Infiltrating Ductal,[Not Available],Positive,Positive,Lumpectomy,1+,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,NO,NO,WHITE
2,60,M0,N0 (i-),Stage IIA,T2,Right Upper Outer Quadrant,Sentinel node biopsy alone,Infiltrating Ductal,[Not Available],Positive,Negative,Simple Mastectomy,2+,6,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,0,NO,NO,WHITE
3,60,M0,N1a,Stage IIB,T2,Left Lower Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,Positive,Lumpectomy,2+,28,Close,Post (prior bilateral ovariectomy OR >12 mo si...,2,NO,NO,WHITE
4,50,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,Positive,Lumpectomy,1+,13,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,1,NO,NO,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,60,M0,N1a,Stage IIA,T1c,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,Positive,Lumpectomy,1+,14,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,NO,NO,WHITE
330,50,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],Positive,Positive,Modified Radical Mastectomy,0,44,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,NO,NO,WHITE
331,50,M0,N2a,Stage IIIA,T2,Left Upper Outer Quadrant|Left Lower Outer Qua...,Axillary lymph node dissection alone,Infiltrating Ductal,[Not Available],Negative,Negative,Modified Radical Mastectomy,1+,26,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,7,NO,NO,WHITE
332,40,M0,N1,Stage IIB,T2,Right Lower Inner Quadrant|Right Lower Outer Q...,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,Positive,Modified Radical Mastectomy,3+,23,Negative,Pre (<6 months since LMP AND no prior bilatera...,2,NO,NO,[Not Available]


In [7]:
index = []
for i in range(0, 334):
    index.append(i)
df['index'] = index
df['age_at_initial_pathologic_diagnosis'] = df['age_at_initial_pathologic_diagnosis'].astype(str)
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_at_initial_pathologic_diagnosis,ajcc_cancer_metastasis_stage_code,ajcc_neoplasm_disease_lymph_node_stage,ajcc_neoplasm_disease_stage,ajcc_tumor_stage_code,anatomic_organ_subdivision,axillary_lymph_node_stage_method_type,breast_cancer_optical_measurement_histologic_type,breast_cancer_surgery_margin_status,breast_carcinoma_estrogen_receptor_status,...,breast_carcinoma_surgical_procedure_name,her2_immunohistochemistry_level_result,lymph_node_examined_count,margin_status,menopause_status,number_of_lymphnodes_positive_by_he,pretreatment_history,prior_diagnosis,race,index
0,50,M0,N2a,Stage IIIA,T3,Right Upper Inner Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],Positive,...,Simple Mastectomy,2+,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,NO,NO,WHITE,0
1,60,M0,N2a,Stage IIIA,T2,Right Upper Outer Quadrant,[Not Available],Infiltrating Ductal,[Not Available],Positive,...,Lumpectomy,1+,27,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,4,NO,NO,WHITE,1
2,60,M0,N0 (i-),Stage IIA,T2,Right Upper Outer Quadrant,Sentinel node biopsy alone,Infiltrating Ductal,[Not Available],Positive,...,Simple Mastectomy,2+,6,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,0,NO,NO,WHITE,2
3,60,M0,N1a,Stage IIB,T2,Left Lower Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,...,Lumpectomy,2+,28,Close,Post (prior bilateral ovariectomy OR >12 mo si...,2,NO,NO,WHITE,3
4,50,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,...,Lumpectomy,1+,13,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,1,NO,NO,WHITE,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,60,M0,N1a,Stage IIA,T1c,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,...,Lumpectomy,1+,14,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,NO,NO,WHITE,329
330,50,M0,N1a,Stage IIB,T2,Left Upper Outer Quadrant|Left,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Lobular,[Not Available],Positive,...,Modified Radical Mastectomy,0,44,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,2,NO,NO,WHITE,330
331,50,M0,N2a,Stage IIIA,T2,Left Upper Outer Quadrant|Left Lower Outer Qua...,Axillary lymph node dissection alone,Infiltrating Ductal,[Not Available],Negative,...,Modified Radical Mastectomy,1+,26,Negative,Post (prior bilateral ovariectomy OR >12 mo si...,7,NO,NO,WHITE,331
332,40,M0,N1,Stage IIB,T2,Right Lower Inner Quadrant|Right Lower Outer Q...,Sentinel lymph node biopsy plus axillary disse...,Infiltrating Ductal,[Not Available],Positive,...,Modified Radical Mastectomy,3+,23,Negative,Pre (<6 months since LMP AND no prior bilatera...,2,NO,NO,[Not Available],332


In [8]:
df_dum = pd.get_dummies(df)
df_dum

,index,age_at_initial_pathologic_diagnosis_20,age_at_initial_pathologic_diagnosis_30,age_at_initial_pathologic_diagnosis_40,age_at_initial_pathologic_diagnosis_50,age_at_initial_pathologic_diagnosis_60,age_at_initial_pathologic_diagnosis_70,age_at_initial_pathologic_diagnosis_80,age_at_initial_pathologic_diagnosis_90,ajcc_cancer_metastasis_stage_code_M0,...,number_of_lymphnodes_positive_by_he_35,number_of_lymphnodes_positive_by_he_[Not Available],pretreatment_history_NO,pretreatment_history_YES,prior_diagnosis_NO,prior_diagnosis_YES,race_ASIAN,race_BLACK OR AFRICAN AMERICAN,race_WHITE,race_[Not Available]
0,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
1,1,0,0,0,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
2,2,0,0,0,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
3,3,0,0,0,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4,4,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,329,0,0,0,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
330,330,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
331,331,0,0,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
332,332,0,0,1,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,1


In [9]:
tensor_dum = df_dum.iloc[:, 1:].values
tensor_dum

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 1, 0]], dtype=uint8)

In [10]:
len(tensor_dum)

334

In [11]:
x = []
for i in range(len(tensor_dum)):
    for j in range(i+1, len(tensor_dum)):
        tensor1 = torch.tensor(tensor_dum[i]).to(torch.float32)
        tensor2 = torch.tensor(tensor_dum[j]).to(torch.float32)
        cos = torch.nn.CosineSimilarity(dim=0)
        output = cos(tensor1, tensor2)
        if output >= 0.55:
            yx = []
            yx.append(i)
            yx.append(j)
            x.append(yx)

In [12]:
df_link = pd.DataFrame(x, columns=['node1', 'node2'])
df_link.to_csv('brca_link.txt', sep = '\t', index=False)

In [320]:
df_dum['y'] = y
df_dum

,index,age_at_initial_pathologic_diagnosis_20,age_at_initial_pathologic_diagnosis_30,age_at_initial_pathologic_diagnosis_40,age_at_initial_pathologic_diagnosis_50,age_at_initial_pathologic_diagnosis_60,age_at_initial_pathologic_diagnosis_70,age_at_initial_pathologic_diagnosis_80,age_at_initial_pathologic_diagnosis_90,ajcc_cancer_metastasis_stage_code_M0,...,number_of_lymphnodes_positive_by_he_[Not Available],pretreatment_history_NO,pretreatment_history_YES,prior_diagnosis_NO,prior_diagnosis_YES,race_ASIAN,race_BLACK OR AFRICAN AMERICAN,race_WHITE,race_[Not Available],y
0,0,0,0,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
1,1,0,0,0,0,1,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
2,2,0,0,0,0,1,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
3,3,0,0,0,0,1,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
4,4,0,0,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,329,0,0,0,0,1,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
330,330,0,0,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
331,331,0,0,0,1,0,0,0,0,1,...,0,1,0,1,0,0,0,1,0,LIVING
332,332,0,0,1,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,1,LIVING


In [321]:
df_dum.to_csv('brca_feature.txt', sep = '\t', index=False)

In [3]:
def load_data(path="/workspace/src/IgAN/gcn/brca/", dataset="brca"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = list(range(159)) + list(range(212, 265))
    idx_val = range(159, 212)
    idx_test = range(265, 334)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    return adj, features, labels, idx_train, idx_val, idx_test

In [4]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [5]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [6]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [7]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [8]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        global weight
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            if epoch == 9:
                weight = output
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [9]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1)
        self.gc2 = GraphConvolution(nhid1, nhid2)
        self.gc3 = GraphConvolution(nhid2, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.gc2(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc3(x, adj)
        return F.log_softmax(x, dim=1)

In [26]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid1)
        self.gc2 = GraphConvolution(nhid1, nhid2)
        self.gc3 = GraphConvolution(nhid2, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        # global l0_features 
        # global l1_features 
        # global l2_features 
        # global l3_features 
        # global l0_weight
        # global l1_weight 
        # global l2_weight
        # global l3_weight 
        # l0_features = x
        # l0_weight = weight
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        # if epoch == 99:
        #     l1_features = x
        #     l1_weight = weight
        #     print("First Layer")
        #     print("Node Feature Matrix", l1_features)
        #     print("Weight", l1_weight, "\n")
        x = F.relu(self.gc2(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        # if epoch == 99:
        #     l2_features = x
        #     l2_weight = weight
        #     print("Second Layer")
        #     print("Node Feature Matrix", l2_features)
        #     print("Weight", l2_weight, "\n")
        x = self.gc3(x, adj)
        # if epoch == 99:
        #     l3_features = x
        #     l3_weight = weight
        #     print("Third Layer")
        #     print("Node Feature Matrix", l3_features)
        #     print("Weight", l3_weight)
        #     print("Adjacency Matrix", adj, "\n")
        return F.log_softmax(x, dim=1)

In [27]:
# Training settings
args = easydict.EasyDict({
    "no-cuda": False,
    "fastmode": False,
    "seed": 42,
    "epoch": 100,
    "lr": 0.01,
    "weight_decay": 5e-4,
    "hidden": 16,
    "dropout": 0.5
})
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [28]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid1=args.hidden,
            nhid2=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

Loading brca dataset...


In [29]:
model

GCN(
  (gc1): GraphConvolution (198 -> 16)
  (gc2): GraphConvolution (16 -> 16)
  (gc3): GraphConvolution (16 -> 2)
)

In [30]:
print('adj :', adj.shape)
print('features :', features.shape)
print('labels :', labels.shape)
print('idx :', idx_train.shape, idx_val.shape, idx_test.shape)

adj : torch.Size([334, 334])
features : torch.Size([334, 198])
labels : torch.Size([334])
idx : torch.Size([212]) torch.Size([53]) torch.Size([69])


In [31]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if (((epoch+1) % 10) == 0) :
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [32]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [33]:
# Train model
t_total = time.time()
for epoch in range(args.epoch):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0010 loss_train: 0.3437 acc_train: 0.9104 loss_val: 0.8358 acc_val: 0.6038 time: 0.0082s
Epoch: 0020 loss_train: 0.3209 acc_train: 0.9104 loss_val: 1.2166 acc_val: 0.6038 time: 0.0081s
Epoch: 0030 loss_train: 0.2965 acc_train: 0.9104 loss_val: 1.0315 acc_val: 0.6038 time: 0.0081s
Epoch: 0040 loss_train: 0.3044 acc_train: 0.9104 loss_val: 0.9459 acc_val: 0.6038 time: 0.0081s
Epoch: 0050 loss_train: 0.2988 acc_train: 0.9104 loss_val: 1.0138 acc_val: 0.6038 time: 0.0081s
Epoch: 0060 loss_train: 0.3040 acc_train: 0.9104 loss_val: 1.0037 acc_val: 0.6038 time: 0.0081s
Epoch: 0070 loss_train: 0.3016 acc_train: 0.9104 loss_val: 0.9840 acc_val: 0.6038 time: 0.0082s
Epoch: 0080 loss_train: 0.2950 acc_train: 0.9104 loss_val: 1.0234 acc_val: 0.6038 time: 0.0081s
Epoch: 0090 loss_train: 0.2949 acc_train: 0.9104 loss_val: 0.9775 acc_val: 0.6038 time: 0.0081s
Epoch: 0100 loss_train: 0.2956 acc_train: 0.9104 loss_val: 0.9726 acc_val: 0.6038 time: 0.0081s
Optimization Finished!
Total time elapse

In [34]:
# Testing
test()

Test set results: loss= 0.5547 accuracy= 0.7971
